Many examples are extensions of https://github.com/PacktPublishing/Interactive-Dashboards-and-Data-Apps-with-Plotly-and-Dash
which are from [Interactive Dashboards and Data Apps with Plotly and Dash](https://www.packtpub.com/product/interactive-dashboards-and-data-apps-with-plotly-and-dash/9781800568914?utm_source=github&utm_medium=repository&utm_campaign=9781800568914)

In [25]:
import intersystems_iris.dbapi._DBAPI as dbapi
from dotenv import load_dotenv
load_dotenv()
import os

# See example from https://github.com/intersystems-community/intersystems-irispython
# Is some issues with the other driver, so use this one.

password = os.getenv("SQL_PASSWORD")
user = os.getenv("SQL_USERNAME")
host = os.getenv("SQL_SERVER")
namespace = os.getenv("SQL_NAMESPACE")
port = os.getenv("SQL_PORT")
if isinstance(port, str):
    port = int(port)

config = {
    "hostname": host,
    "port": port,
    "namespace": namespace,
    "username": user,
    "password": password,
}

try:
    conn = dbapi.connect(**config)
    print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")

# create a cursor
cursor = conn.cursor()
cursor


Connection successful!


In [26]:
import pandas as pd

sql = """
      select ResourceType, count(*) from HSFHIR_X0001_R.Rsrc group by ResourceType
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
df

,ResourceType,Aggregate_2
0,Binary,100
1,DiagnosticReport,8
2,DocumentReference,8
3,Encounter,12
4,Observation,2
5,Organization,15
6,Patient,10
7,ServiceRequest,11
8,Specimen,3


In [32]:
sql = """
      select dr.ID1,CDR.GetJSON(ResourceString,'basedOn'),$PIECE(CDR.GetFHIRPathOne(ResourceString,'basedOn[0].reference','DiagnosticReport'),'/',2) ServiceRequest,sr.requester_IdentifierValue from HSFHIR_X0001_S.DiagnosticReport dr
  join HSFHIR_X0001_R.Rsrc resource on resource.ResourceType = 'DiagnosticReport'
          and resource.Key = dr.Key
  left outer join HSFHIR_X0001_S.ServiceRequest sr on CDR.GetFHIRPathOne(ResourceString,'basedOn[0].reference','DiagnosticReport')=sr.key
      """

cursor.execute(sql)
data = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
df = pd.DataFrame(data, columns=column_names)
df

,ID1,Expression_2,ServiceRequest,requester_IdentifierValue
0,1,"[{""identifier"":{""assigner"":{""identifier"":{""sys...",None,None
1,2,"[{""identifier"":{""assigner"":{""identifier"":{""sys...",15,RX1
2,3,"[{""identifier"":{""assigner"":{""identifier"":{""sys...",None,None
3,4,"[{""identifier"":{""assigner"":{""identifier"":{""sys...",60,R0A
4,5,"[{""identifier"":{""assigner"":{""identifier"":{""sys...",74,R0A
5,6,"[{""identifier"":{""assigner"":{""identifier"":{""sys...",87,RWJ
6,7,"[{""identifier"":{""assigner"":{""identifier"":{""sys...",133,REP
7,8,"[{""identifier"":{""assigner"":{""identifier"":{""sys...",118,None
